In [1]:
!yarn --daemon start resourcemanager
!yarn --daemon start proxyserver

In [2]:
!yarn --daemon start nodemanager

In [104]:
!hadoop fs -mkdir -p /media/notebooks/Modulo1/prueba

In [105]:
import os
os.chdir("/media/notebooks/Modulo1/prueba")

In [109]:
!cp /dataset/texto.txt /media/notebooks/Modulo1/prueba

In [110]:
!ls

mapper.py  reducer.py  salida  texto.txt


In [115]:
%%writefile mapper.py
#!/usr/bin/python3
# coding=utf-8
import sys

#Leemos cada línea de la entrada estándar
for line in sys.stdin:  
  #Dividimos en palabras cada línea
  words = line.split()  
  #Iteramos sobre cada palabra contenida en words
  for word in words:    
  	#Escribimos por salida estándar el par <Clave,Valor>
	#En este caso la Clave será la palabra en cuestión y el valor será 1
	print '%s\t%s' % (word,1)

Overwriting mapper.py


In [116]:
%%writefile reducer.py
#!/usr/bin/python3
# coding=utf-8
import sys
	
curr_word = None
curr_count = 0
	
# Procesamos cada Clave-Valor que produce la función map
for line in sys.stdin:
	
  #Dividimos la clave y el valor
  word, count = line.split('\t')
	
  #Convertimos a entero el valor
  count = int(count)

  #Si la palabra actual es la misma que la previa, incrementamos el contador
  #en otro caso imprimimos la palabra en cuestión con el contador
  if word == curr_word:
    curr_count += count
  else:
    if curr_word:   
      print '%s\t%s' % (curr_word, curr_count)
			
    #Actualizamos las variables a la nueva palabra
    curr_word = word
    curr_count = count
	
#Imprimimos la última palabra
if curr_word == word:
  print '%s\t%s' % (curr_word, curr_count)

Overwriting reducer.py


In [31]:
!ls -lh

total 4.0K
-rw-r--r-- 1 root root 392 Nov 15 21:57 mapper.py
-rw-r--r-- 1 root root 733 Nov 15 21:57 reducer.py


In [49]:
!cat /dataset/texto.txt | python mapper.py | sort | python reducer.py > salida

In [50]:
!tail -60000 salida | head -10

Esto	2
contador	1
de	3
ejemplo	2
es	2
mapreduce.	1
palabras.	1
programación	1
un	2


In [121]:
!ls .

mapper.py  reducer.py  salida  texto.txt


In [122]:
!mapred streaming \
  -input /media/notebooks/Modulo1/prueba/texto.txt \
  -output /media/notebooks/Modulo1/prueba/output2 \
  -file ./reducer.py \
  -file ./mapper.py \
  -mapper mapper.py \
  -reducer reducer.py

2022-11-15 22:46:45,414 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [./reducer.py, ./mapper.py] [/app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar] /tmp/streamjob1472128479229428702.jar tmpDir=null
2022-11-15 22:46:46,293 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmaster/172.18.0.5:8032
2022-11-15 22:46:46,452 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmaster/172.18.0.5:8032
2022-11-15 22:46:46,651 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1668543377787_0007
2022-11-15 22:46:47,069 INFO mapred.FileInputFormat: Total input files to process : 1
2022-11-15 22:46:47,163 INFO mapreduce.JobSubmitter: number of splits:2
2022-11-15 22:46:47,341 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1668543377787_0007
2022-11-15 22:46:47,341 INFO mapred

2022-11-15 22:47:14,131 INFO mapreduce.Job: Counters: 14
	Job Counters 
		Failed map tasks=7
		Killed map tasks=1
		Killed reduce tasks=1
		Launched map tasks=8
		Other local map tasks=6
		Data-local map tasks=2
		Total time spent by all maps in occupied slots (ms)=25284
		Total time spent by all reduces in occupied slots (ms)=0
		Total time spent by all map tasks (ms)=25284
		Total vcore-milliseconds taken by all map tasks=25284
		Total megabyte-milliseconds taken by all map tasks=25890816
	Map-Reduce Framework
		CPU time spent (ms)=0
		Physical memory (bytes) snapshot=0
		Virtual memory (bytes) snapshot=0
2022-11-15 22:47:14,131 ERROR streaming.StreamJob: Job not successful!
Streaming Command Failed!
